In [1]:
import warnings
warnings.filterwarnings("ignore")
import ea_wrangle as w

import pandas as pd
import numpy as np

#splits
from sklearn.model_selection import train_test_split

#visualization
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns

#scaling
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import precision_score, accuracy_score, recall_score, classification_report

#model
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.linear_model import LogisticRegression


# Acquire

In [2]:
merged_data = w.left_join_csv('austin_animal_outcomes.csv', 'austin_animal_intakes.csv', 'merged_data.csv')

In [3]:
merged_data 

,Animal ID,Name_x,DateTime_x,MonthYear_x,Date of Birth,Outcome Type,Outcome Subtype,Animal Type_x,Sex upon Outcome,Age upon Outcome,...,DateTime_y,MonthYear_y,Found Location,Intake Type,Intake Condition,Animal Type_y,Sex upon Intake,Age upon Intake,Breed_y,Color_y
0,A794011,Chunk,05/08/2019 06:20:00 PM,May 2019,05/02/2017,Rto-Adopt,NaN,Cat,Neutered Male,2 years,...,05/02/2019 04:51:00 PM,May 2019,Austin (TX),Owner Surrender,Normal,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
1,A776359,Gizmo,07/18/2018 04:02:00 PM,Jul 2018,07/12/2017,Adoption,NaN,Dog,Neutered Male,1 year,...,07/12/2018 12:46:00 PM,July 2018,7201 Levander Loop in Austin (TX),Stray,Normal,Dog,Intact Male,1 year,Chihuahua Shorthair Mix,White/Brown
2,A821648,NaN,08/16/2020 11:38:00 AM,Aug 2020,08/16/2019,Euthanasia,NaN,Other,Unknown,1 year,...,08/16/2020 10:10:00 AM,August 2020,Armadillo Rd And Clubway Ln in Austin (TX),Wildlife,Sick,Other,Unknown,1 year,Raccoon,Gray
3,A720371,Moose,02/13/2016 05:59:00 PM,Feb 2016,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,...,02/08/2016 11:05:00 AM,February 2016,Dove Dr And E Stassney in Austin (TX),Stray,Normal,Dog,Intact Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
4,A720371,Moose,02/13/2016 05:59:00 PM,Feb 2016,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,...,02/15/2016 10:37:00 AM,February 2016,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197199,A884455,NaN,07/10/2023 07:52:00 AM,Jul 2023,07/03/2021,Transfer,Snr,Cat,Unknown,2 years,...,07/03/2023 11:55:00 AM,July 2023,8104 Taza Trail in Austin (TX),Stray,Normal,Cat,Unknown,2 years,Domestic Shorthair,Orange Tabby
197200,A884621,NaN,07/06/2023 06:53:00 PM,Jul 2023,07/06/2021,Transfer,Partner,Dog,Intact Male,2 years,...,07/06/2023 07:05:00 AM,July 2023,Rundberg And Ih 35 in Austin (TX),Stray,Injured,Dog,Intact Male,2 years,Pit Bull,Fawn/White
197201,A884354,NaN,07/10/2023 07:51:00 AM,Jul 2023,07/01/2020,Transfer,Snr,Cat,Intact Male,3 years,...,07/01/2023 04:06:00 PM,July 2023,1710 E Oltorf Rd Apt 814 in Austin (TX),Stray,Normal,Cat,Intact Male,3 years,Domestic Shorthair,Cream Tabby
197202,A878160,Lady,07/10/2023 10:54:00 AM,Jul 2023,04/07/2021,Transfer,Partner,Dog,Intact Female,2 years,...,04/07/2023 03:11:00 PM,April 2023,4500 East William Cannon in Austin (TX),Stray,Normal,Dog,Intact Female,2 years,Siberian Husky Mix,Gray/Black


In [4]:
df = merged_data
df.head()

,Animal ID,Name_x,DateTime_x,MonthYear_x,Date of Birth,Outcome Type,Outcome Subtype,Animal Type_x,Sex upon Outcome,Age upon Outcome,...,DateTime_y,MonthYear_y,Found Location,Intake Type,Intake Condition,Animal Type_y,Sex upon Intake,Age upon Intake,Breed_y,Color_y
0,A794011,Chunk,05/08/2019 06:20:00 PM,May 2019,05/02/2017,Rto-Adopt,NaN,Cat,Neutered Male,2 years,...,05/02/2019 04:51:00 PM,May 2019,Austin (TX),Owner Surrender,Normal,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
1,A776359,Gizmo,07/18/2018 04:02:00 PM,Jul 2018,07/12/2017,Adoption,NaN,Dog,Neutered Male,1 year,...,07/12/2018 12:46:00 PM,July 2018,7201 Levander Loop in Austin (TX),Stray,Normal,Dog,Intact Male,1 year,Chihuahua Shorthair Mix,White/Brown
2,A821648,NaN,08/16/2020 11:38:00 AM,Aug 2020,08/16/2019,Euthanasia,NaN,Other,Unknown,1 year,...,08/16/2020 10:10:00 AM,August 2020,Armadillo Rd And Clubway Ln in Austin (TX),Wildlife,Sick,Other,Unknown,1 year,Raccoon,Gray
3,A720371,Moose,02/13/2016 05:59:00 PM,Feb 2016,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,...,02/08/2016 11:05:00 AM,February 2016,Dove Dr And E Stassney in Austin (TX),Stray,Normal,Dog,Intact Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
4,A720371,Moose,02/13/2016 05:59:00 PM,Feb 2016,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,...,02/15/2016 10:37:00 AM,February 2016,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff


In [5]:
df = w.get_prep_aa(df)

In [6]:
df.columns

Index(['name', 'outcome', 'dob', 'intake_type', 'intake_datetime',
       'outcome_datetime', 'intake_condition', 'intake_age', 'outcome_age',
       'species', 'intake_sex', 'breed', 'mixed', 'two_breeds', 'pure_breeds',
       'primary_color', 'is_tabby', 'mix_color'],
      dtype='object')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185240 entries, 0 to 197203
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   name              185240 non-null  object
 1   outcome           185240 non-null  object
 2   dob               185240 non-null  object
 3   intake_type       185240 non-null  object
 4   intake_datetime   185240 non-null  object
 5   outcome_datetime  185240 non-null  object
 6   intake_condition  185240 non-null  object
 7   intake_age        185240 non-null  object
 8   outcome_age       185240 non-null  object
 9   species           185240 non-null  object
 10  intake_sex        185240 non-null  object
 11  breed             185240 non-null  object
 12  mixed             185240 non-null  object
 13  two_breeds        185240 non-null  object
 14  pure_breeds       185240 non-null  object
 15  primary_color     185240 non-null  object
 16  is_tabby          185240 non-null  int

df['intake_datetime'].dt.date

df['breed'][df['breed'].str.contains('mix', case=False)]

In [8]:
df.head(10)

,name,outcome,dob,intake_type,intake_datetime,outcome_datetime,intake_condition,intake_age,outcome_age,species,intake_sex,breed,mixed,two_breeds,pure_breeds,primary_color,is_tabby,mix_color
0,chunk,rto-adopt,2017-05-02,owner surrender,2019-05-02 16:51:00,2019-05-08 18:20:00,normal,24,24,cat,neutered male,domestic shorthair mix,1,0,nan,brown,1,1
1,gizmo,adoption,2017-07-12,stray,2018-07-12 12:46:00,2018-07-18 16:02:00,normal,12,12,dog,intact male,chihuahua shorthair mix,1,0,nan,white,0,1
3,moose,adoption,2015-10-08,stray,2016-02-08 11:05:00,2016-02-13 17:59:00,normal,4,4,dog,intact male,anatol shepherd/labrador retriever,0,1,nan,buff,0,0
4,moose,adoption,2015-10-08,owner surrender,2016-02-15 10:37:00,2016-02-13 17:59:00,normal,4,4,dog,neutered male,anatol shepherd/labrador retriever,0,1,nan,buff,0,0
5,nan,transfer,2014-03-12,stray,2014-03-18 07:11:00,2014-03-18 11:47:00,medical attention,0,0,cat,intact male,domestic shorthair mix,1,0,nan,orange,1,0
6,princess,adoption,2013-03-24,owner surrender,2018-06-13 12:55:00,2020-10-05 14:37:00,normal,63,91,dog,spayed female,chihuahua shorthair mix,1,0,nan,brown,0,0
7,princess,adoption,2013-03-24,owner surrender,2020-10-05 11:20:00,2020-10-05 14:37:00,normal,91,91,dog,spayed female,chihuahua shorthair mix,1,0,nan,brown,0,0
8,quentin,adoption,2018-03-01,owner surrender,2020-03-01 14:19:00,2020-05-06 07:59:00,normal,24,26,dog,neutered male,american foxhound/labrador retriever,0,1,nan,white,0,1
9,*lalo,adoption,2022-05-04,stray,2022-06-23 15:09:00,2022-07-19 15:53:00,normal,1,2,cat,intact male,domestic shorthair,0,0,nan,brown,1,0
10,*donatello,adoption,2014-08-01,stray,2014-10-08 16:39:00,2014-10-18 18:52:00,normal,2,2,cat,intact male,domestic shorthair mix,1,0,nan,black,0,0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185240 entries, 0 to 197203
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   name              185240 non-null  object
 1   outcome           185240 non-null  object
 2   dob               185240 non-null  object
 3   intake_type       185240 non-null  object
 4   intake_datetime   185240 non-null  object
 5   outcome_datetime  185240 non-null  object
 6   intake_condition  185240 non-null  object
 7   intake_age        185240 non-null  object
 8   outcome_age       185240 non-null  object
 9   species           185240 non-null  object
 10  intake_sex        185240 non-null  object
 11  breed             185240 non-null  object
 12  mixed             185240 non-null  object
 13  two_breeds        185240 non-null  object
 14  pure_breeds       185240 non-null  object
 15  primary_color     185240 non-null  object
 16  is_tabby          185240 non-null  int

# Prepare

pull date form 

In [10]:
#make intake and outcome datetimes into just the dates, make intake and outcome ages without times

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185240 entries, 0 to 197203
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   name              185240 non-null  object
 1   outcome           185240 non-null  object
 2   dob               185240 non-null  object
 3   intake_type       185240 non-null  object
 4   intake_datetime   185240 non-null  object
 5   outcome_datetime  185240 non-null  object
 6   intake_condition  185240 non-null  object
 7   intake_age        185240 non-null  object
 8   outcome_age       185240 non-null  object
 9   species           185240 non-null  object
 10  intake_sex        185240 non-null  object
 11  breed             185240 non-null  object
 12  mixed             185240 non-null  object
 13  two_breeds        185240 non-null  object
 14  pure_breeds       185240 non-null  object
 15  primary_color     185240 non-null  object
 16  is_tabby          185240 non-null  int

df['mix_breeds'] = df['breed'][df['breed'].str.contains('mix', case=False)]

df['two_breeds'] = df['breed'][df['breed'].str.contains('/', case=False)]

In [12]:
df.head()

,name,outcome,dob,intake_type,intake_datetime,outcome_datetime,intake_condition,intake_age,outcome_age,species,intake_sex,breed,mixed,two_breeds,pure_breeds,primary_color,is_tabby,mix_color
0,chunk,rto-adopt,2017-05-02,owner surrender,2019-05-02 16:51:00,2019-05-08 18:20:00,normal,24,24,cat,neutered male,domestic shorthair mix,1,0,nan,brown,1,1
1,gizmo,adoption,2017-07-12,stray,2018-07-12 12:46:00,2018-07-18 16:02:00,normal,12,12,dog,intact male,chihuahua shorthair mix,1,0,nan,white,0,1
3,moose,adoption,2015-10-08,stray,2016-02-08 11:05:00,2016-02-13 17:59:00,normal,4,4,dog,intact male,anatol shepherd/labrador retriever,0,1,nan,buff,0,0
4,moose,adoption,2015-10-08,owner surrender,2016-02-15 10:37:00,2016-02-13 17:59:00,normal,4,4,dog,neutered male,anatol shepherd/labrador retriever,0,1,nan,buff,0,0
5,nan,transfer,2014-03-12,stray,2014-03-18 07:11:00,2014-03-18 11:47:00,medical attention,0,0,cat,intact male,domestic shorthair mix,1,0,nan,orange,1,0


In [13]:
#df['pure_breed'] = df['breed'][df['breed'].str.contains('mix|/', case=False)]

df['breed'].fillna('', inplace=True)

df['pure_bred'] = df[df['breed'].astype(str).str.contains('mix|/', case=False) == False].breed

In [14]:
counts = df['breed'].value_counts()

mask = df['breed'].isin(counts[counts > 200].index)

df = df[mask]

In [15]:
df.breed.value_counts()

domestic shorthair mix                   36215
pit bull mix                             16144
domestic shorthair                       15872
labrador retriever mix                   11605
chihuahua shorthair mix                   8254
                                         ...  
chinese sharpei/rhod ridgeback               1
pomeranian/american eskimo                   1
miniature pinscher/smooth fox terrier        1
collie rough/pembroke welsh corgi            1
catahoula/black/tan hound                    1
Name: breed, Length: 2620, dtype: int64

df['mix_dummy'] = np.where(df['breed'].str.contains('mix', case=False, na=False), 1, 0)
df['two_breeds'] = np.where(df['breed'].str.contains('/', case=False, na=False), 1, 0)
df['pure_bred'] = np.where(df['breed'].isin(['/', 'mix']), 1, 0)

In [16]:
df.head(20)

,name,outcome,dob,intake_type,intake_datetime,outcome_datetime,intake_condition,intake_age,outcome_age,species,intake_sex,breed,mixed,two_breeds,pure_breeds,primary_color,is_tabby,mix_color
0,chunk,rto-adopt,2017-05-02,owner surrender,2019-05-02 16:51:00,2019-05-08 18:20:00,normal,24,24,cat,neutered male,domestic shorthair mix,1,0,nan,brown,1,1
1,gizmo,adoption,2017-07-12,stray,2018-07-12 12:46:00,2018-07-18 16:02:00,normal,12,12,dog,intact male,chihuahua shorthair mix,1,0,nan,white,0,1
3,moose,adoption,2015-10-08,stray,2016-02-08 11:05:00,2016-02-13 17:59:00,normal,4,4,dog,intact male,anatol shepherd/labrador retriever,0,1,nan,buff,0,0
4,moose,adoption,2015-10-08,owner surrender,2016-02-15 10:37:00,2016-02-13 17:59:00,normal,4,4,dog,neutered male,anatol shepherd/labrador retriever,0,1,nan,buff,0,0
5,nan,transfer,2014-03-12,stray,2014-03-18 07:11:00,2014-03-18 11:47:00,medical attention,0,0,cat,intact male,domestic shorthair mix,1,0,nan,orange,1,0
6,princess,adoption,2013-03-24,owner surrender,2018-06-13 12:55:00,2020-10-05 14:37:00,normal,63,91,dog,spayed female,chihuahua shorthair mix,1,0,nan,brown,0,0
7,princess,adoption,2013-03-24,owner surrender,2020-10-05 11:20:00,2020-10-05 14:37:00,normal,91,91,dog,spayed female,chihuahua shorthair mix,1,0,nan,brown,0,0
8,quentin,adoption,2018-03-01,owner surrender,2020-03-01 14:19:00,2020-05-06 07:59:00,normal,24,26,dog,neutered male,american foxhound/labrador retriever,0,1,nan,white,0,1
9,*lalo,adoption,2022-05-04,stray,2022-06-23 15:09:00,2022-07-19 15:53:00,normal,1,2,cat,intact male,domestic shorthair,0,0,nan,brown,1,0
10,*donatello,adoption,2014-08-01,stray,2014-10-08 16:39:00,2014-10-18 18:52:00,normal,2,2,cat,intact male,domestic shorthair mix,1,0,nan,black,0,0


df['pure_bred'] = np.where(df['breed'].isin(['/', 'mix']), 1, 0)

In [17]:
df.head(20)

,name,outcome,dob,intake_type,intake_datetime,outcome_datetime,intake_condition,intake_age,outcome_age,species,intake_sex,breed,mixed,two_breeds,pure_breeds,primary_color,is_tabby,mix_color
0,chunk,rto-adopt,2017-05-02,owner surrender,2019-05-02 16:51:00,2019-05-08 18:20:00,normal,24,24,cat,neutered male,domestic shorthair mix,1,0,nan,brown,1,1
1,gizmo,adoption,2017-07-12,stray,2018-07-12 12:46:00,2018-07-18 16:02:00,normal,12,12,dog,intact male,chihuahua shorthair mix,1,0,nan,white,0,1
3,moose,adoption,2015-10-08,stray,2016-02-08 11:05:00,2016-02-13 17:59:00,normal,4,4,dog,intact male,anatol shepherd/labrador retriever,0,1,nan,buff,0,0
4,moose,adoption,2015-10-08,owner surrender,2016-02-15 10:37:00,2016-02-13 17:59:00,normal,4,4,dog,neutered male,anatol shepherd/labrador retriever,0,1,nan,buff,0,0
5,nan,transfer,2014-03-12,stray,2014-03-18 07:11:00,2014-03-18 11:47:00,medical attention,0,0,cat,intact male,domestic shorthair mix,1,0,nan,orange,1,0
6,princess,adoption,2013-03-24,owner surrender,2018-06-13 12:55:00,2020-10-05 14:37:00,normal,63,91,dog,spayed female,chihuahua shorthair mix,1,0,nan,brown,0,0
7,princess,adoption,2013-03-24,owner surrender,2020-10-05 11:20:00,2020-10-05 14:37:00,normal,91,91,dog,spayed female,chihuahua shorthair mix,1,0,nan,brown,0,0
8,quentin,adoption,2018-03-01,owner surrender,2020-03-01 14:19:00,2020-05-06 07:59:00,normal,24,26,dog,neutered male,american foxhound/labrador retriever,0,1,nan,white,0,1
9,*lalo,adoption,2022-05-04,stray,2022-06-23 15:09:00,2022-07-19 15:53:00,normal,1,2,cat,intact male,domestic shorthair,0,0,nan,brown,1,0
10,*donatello,adoption,2014-08-01,stray,2014-10-08 16:39:00,2014-10-18 18:52:00,normal,2,2,cat,intact male,domestic shorthair mix,1,0,nan,black,0,0


In [18]:
df.shape

(185240, 18)

# Question and Hypothesis Statement

# Conclusion

# Question and Hypothesis Statement

# Conclusion

# Question and Hypothesis Statement

# Conclusion